In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import nearest_points


In [2]:
# Workaround to fix chrome issue where folium won't plot maps with a large number of layers
# See comment by dstein64 at: https://github.com/python-visualization/folium/issues/812

import base64
def _repr_html_(self, **kwargs):
    html = base64.b64encode(self.render(**kwargs).encode('utf8')).decode('utf8')
    onload = (
        'this.contentDocument.open();'
        'this.contentDocument.write(atob(this.getAttribute(\'data-html\')));'
        'this.contentDocument.close();'
    )
    if self.height is None:
        iframe = (
            '<div style="width:{width};">'
            '<div style="position:relative;width:100%;height:0;padding-bottom:{ratio};">'
            '<iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;'
            'border:none !important;" '
            'data-html={html} onload="{onload}" '
            'allowfullscreen webkitallowfullscreen mozallowfullscreen>'
            '</iframe>'
            '</div></div>').format
        iframe = iframe(html=html, onload=onload, width=self.width, ratio=self.ratio)
    else:
        iframe = ('<iframe src="about:blank" width="{width}" height="{height}"'
                  'style="border:none !important;" '
                  'data-html={html} onload="{onload}" '
                  '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
                  '</iframe>').format
        iframe = iframe(html=html, onload=onload, width=self.width, height=self.height)
    return iframe

folium.branca.element.Figure._repr_html_ = _repr_html_


In [54]:
filepath = '/Volumes/ZHANG MBP/Strava data/Datasets_1/Strava Metro Assorted/safeguarded-release-strava-metro-glasgow-2015-all-extract/'

glasgow_nodes_2015 = gpd.read_file(filepath + 'glasgow_1k_2015_nodes.shp')
glasgow_edges_2015 = gpd.read_file(filepath + 'glasgow_1k_edges_2015.shp')
glasgow_edges_polygon = gpd.read_file(filepath + 'glasgow_1k_od_2015_polygons.shp')

node_alignment = pd.read_csv(filepath + 'glasgow_2015_ride_node_alignment_hourly.csv')
total_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_nodes_rollup_total.csv')
month_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_node_rollup_month_total.csv')
hourly_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_node_alignment_hourly.csv')

edge_alignment = pd.read_csv(filepath + 'glasgow_2015_ride_edge_alignment_hourly.csv')
# total_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_nodes_rollup_total.csv')
month_edge_count = pd.read_csv(filepath + 'glasgow_2015_ride_edge_rollup_month_total.csv')
hourly_edge_count = pd.read_csv(filepath + 'glasgow_2015_ride_edge_rollup_total.csv')


In [7]:
place = 'Glasgow City, Scotland, UK'
city = ox.gdf_from_place(place)


In [72]:
total_node_count[~pd.isna(total_node_count.pct_0_0)]

,node_id,athcnt_0,actcnt_0,cmtcnt_0,pct_0_0,pct_025_0,pct_05_0,pct_075_0,pct_1_0,athcnt_1,...,pct_075_4,pct_1_4,athcnt,actcnt,cmtcnt,pct_0,pct_025,pct_05,pct_075,pct_1
14,119414,10,10,10,1.0,1.00,2.5,3.00,12.0,70,...,4.0,43.0,225,1070,645,1,1.0,1.0,3.0,775
22,119550,5,10,10,1.0,1.00,3.5,9.25,18.0,30,...,5.5,12.0,100,315,180,1,1.0,1.0,4.0,132
36,119925,5,10,10,1.0,1.00,1.0,5.75,9.0,30,...,5.0,9.0,95,305,175,1,1.0,1.0,4.0,15
58,120195,5,5,5,1.0,4.75,6.0,6.50,8.0,255,...,2.0,12.0,1160,3950,2840,0,1.0,1.0,3.0,231
76,120372,10,40,20,1.0,4.00,8.0,9.00,12.0,280,...,7.0,11.0,800,3080,1860,1,1.0,1.0,7.0,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28999,219408,5,5,5,1.0,1.00,1.0,1.00,1.0,125,...,2.0,5.0,385,2100,1270,1,1.0,1.0,2.0,12
29000,219411,5,20,15,1.0,1.00,1.0,1.00,2.0,100,...,3.0,4.0,300,1900,1340,1,1.0,1.0,3.0,9
29003,219447,5,20,15,1.0,1.00,1.0,1.00,3.0,100,...,2.0,5.0,295,1885,1330,1,1.0,1.0,2.0,17
29008,219539,5,20,15,1.0,1.00,1.0,1.00,2.0,100,...,1.0,5.0,305,1910,1345,1,1.0,1.0,2.0,10


In [66]:
glasgow_nodes_2015[glasgow_nodes_2015.ID == 119104]

,ID,geometry
16,119104.0,MULTIPOINT (-4.40491 55.91324)


In [51]:
glasgow_nodes_2015.head()

,ID,geometry
0,118831.0,MULTIPOINT (-4.40579 55.91053)
1,118842.0,MULTIPOINT (-4.40575 55.91091)
2,118845.0,MULTIPOINT (-4.40573 55.91112)
3,118910.0,MULTIPOINT (-4.40554 55.91052)
4,118926.0,MULTIPOINT (-4.40549 55.90910)


In [ ]:
m = folium.Map([city.geometry.centroid.y, city.geometry.centroid.x],
               zoom_start=11,
               tiles="CartoDb dark_matter")

style_city = {'color':'#ebc923 ', 'fillColor': '#ebc923 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city, style_function=lambda x: style_city).add_to(m)

# style_nodes = {'color':'#ebc923 ', 'fillColor': '#ebc923 ', 'weight':'1', 'fillOpacity' : 0.1}
# folium.GeoJson(glasgow_nodes_2015.iloc[10000:10050], style_function=lambda x: style_nodes).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}

folium.Choropleth(glasgow_edges_polygon.iloc[:1], fill_color = '#1FFD09',
                      fill_opacity = 1 ).add_to(m)
    
m

44781